# Лабораторная работа №5: Проведение исследований с градиентным бустингом

## 1. Выбор начальных условий

Был проведен в ЛР №1.

In [11]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [13]:
date_fruit_path = "/kaggle/input/date-fruit-datasets"

print("Path to Date Fruit dataset files:", date_fruit_path)

# Проверка содержимого папки
if os.path.exists(date_fruit_path):
    files = os.listdir(date_fruit_path)
    print("Files:", files)
else:
    print("Not found")

Path to Date Fruit dataset files: /kaggle/input/date-fruit-datasets
Files: ['Date_Fruit_Datasets']


In [14]:
# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

# Проверка данных
date_fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [15]:
concrete_strength_path = "/kaggle/input/concrete-compressive-strength"

print("Путь до датасета Concrete Compressive Strength:", concrete_strength_path)

if os.path.exists(concrete_strength_path):
    files = os.listdir(concrete_strength_path)
    print("Содержание:", files)
else:
    print("Не найден")

Путь до датасета Concrete Compressive Strength: /kaggle/input/concrete-compressive-strength
Содержание: ['Concrete Compressive Strength.csv']


In [16]:
# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


## 2. Создание бейзлайна и оценка качества

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

Разделим датасет для классификации на обучающую и тестовую выборки

In [18]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [19]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [20]:
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_classifier.fit(X_train_class, y_train_class)

y_pred_class = gb_classifier.predict(X_test_class)

accuracy_gb = accuracy_score(y_test_class, y_pred_class)
f1_gb = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Gradient Boosting Classifier - Accuracy: {accuracy_gb:.4f}, F1-Score: {f1_gb:.4f}")

Gradient Boosting Classifier - Accuracy: 0.8778, F1-Score: 0.8807


In [22]:
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = gb_regressor.predict(X_test_reg)

rmse_gb = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2_gb = r2_score(y_test_reg, y_pred_reg)

print(f"Gradient Boosting Regressor - RMSE: {rmse_gb:.4f}, R²: {r2_gb:.4f}")

Gradient Boosting Regressor - RMSE: 5.5422, R²: 0.8808


Точность для встроенной в Sklearn модели градиентного бустинга для классификатора, получилась немного хуже, чем для случайного леса (87.8% точности против 92,2%), а для регрессора - на уровне случайного леса. Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Будем перебирать гиперпараметры градиентного бустинга, такие, как n_estimators, learning_rate, max_depth, min_samples_split, min_samples_leaf

In [23]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# === Подбор гиперпараметров для GradientBoostingClassifier ===
print("=== Gradient Boosting Classifier - Hyperparameter Tuning ===")
param_dist_classifier = {
    'n_estimators': [80, 100, 120],
    'learning_rate': [0.05, 0.08, 0.1, 0.15],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0]
}

gb_classifier_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_distributions=param_dist_classifier,
    n_iter=15,              # Увеличили количество комбинаций
    scoring='accuracy',
    cv=3,
    verbose=2,              # Более подробный вывод
    random_state=42,
    n_jobs=-1              # Использование всех ядер процессора
)

gb_classifier_search.fit(X_train_class, y_train_class)

# Лучшие параметры и результат
best_gb_classifier_params = gb_classifier_search.best_params_
best_gb_classifier_score = gb_classifier_search.best_score_

print(f"Лучшие параметры Gradient Boosting Classifier: {best_gb_classifier_params}")
print(f"Лучшая Accuracy (кросс-валидация): {best_gb_classifier_score:.4f}")

# === Подбор гиперпараметров для GradientBoostingRegressor ===
print("\n=== Gradient Boosting Regressor - Hyperparameter Tuning ===")
param_dist_regressor = {
    'n_estimators': [80, 100, 120],
    'learning_rate': [0.05, 0.08, 0.1, 0.15],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'loss': ['squared_error', 'absolute_error']
}

gb_regressor_search = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_distributions=param_dist_regressor,
    n_iter=15,              # Увеличили количество комбинаций
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

gb_regressor_search.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат
best_gb_regressor_params = gb_regressor_search.best_params_
best_gb_regressor_score = -gb_regressor_search.best_score_

print(f"Лучшие параметры Gradient Boosting Regressor: {best_gb_regressor_params}")
print(f"Лучший RMSE (кросс-валидация): {best_gb_regressor_score:.4f}")

# Дополнительная информация
print(f"\nВсего комбинаций для классификатора: {len(param_dist_classifier)}")
print(f"Всего комбинаций для регрессора: {len(param_dist_regressor)}")

=== Gradient Boosting Classifier - Hyperparameter Tuning ===
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Лучшие параметры Gradient Boosting Classifier: {'subsample': 0.8, 'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 4, 'learning_rate': 0.15}
Лучшая Accuracy (кросс-валидация): 0.8704

=== Gradient Boosting Regressor - Hyperparameter Tuning ===
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Лучшие параметры Gradient Boosting Regressor: {'subsample': 0.8, 'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 5, 'loss': 'squared_error', 'learning_rate': 0.15}
Лучший RMSE (кросс-валидация): 4.6814

Всего комбинаций для классификатора: 6
Всего комбинаций для регрессора: 7


Обучим модели с лучшими гиперпараметрами

In [39]:
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
import numpy as np

# GradientBoostingClassifier с подобранными параметрами
print("=== Оценка лучшего Gradient Boosting Classifier ===")
optimized_gb_classifier = GradientBoostingClassifier(
    n_estimators=best_gb_classifier_params['n_estimators'],
    learning_rate=best_gb_classifier_params['learning_rate'],
    max_depth=best_gb_classifier_params['max_depth'],
    min_samples_split=best_gb_classifier_params['min_samples_split'],
    min_samples_leaf=best_gb_classifier_params['min_samples_leaf'],
    subsample=best_gb_classifier_params.get('subsample', 1.0),
    random_state=42
)

# Обучение модели
optimized_gb_classifier.fit(X_train_class, y_train_class)

# Предсказания
y_pred_class = optimized_gb_classifier.predict(X_test_class)
y_pred_prob_class = optimized_gb_classifier.predict_proba(X_test_class)[:, 1]

# Метрики
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Оптимальные параметры модели: {best_gb_classifier_params}")
print(f"Accuracy на тестовой выборке: {accuracy:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")
print(f"Количество деревьев: {best_gb_classifier_params['n_estimators']}")
print(f"Скорость обучения: {best_gb_classifier_params['learning_rate']}")

# GradientBoostingRegressor с подобранными параметрами
print("\n=== Оценка лучшего Gradient Boosting Regressor ===")
optimized_gb_regressor = GradientBoostingRegressor(
    n_estimators=best_gb_regressor_params['n_estimators'],
    learning_rate=best_gb_regressor_params['learning_rate'],
    max_depth=best_gb_regressor_params['max_depth'],
    min_samples_split=best_gb_regressor_params['min_samples_split'],
    min_samples_leaf=best_gb_regressor_params['min_samples_leaf'],
    subsample=best_gb_regressor_params.get('subsample', 1.0),
    loss=best_gb_regressor_params.get('loss', 'squared_error'),  # Безопасное получение параметра
    random_state=42
)

# Обучение модели
optimized_gb_regressor.fit(X_train_reg, y_train_reg)

# Предсказания
y_pred_reg = optimized_gb_regressor.predict(X_test_reg)

# Метрики
mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_reg, y_pred_reg)
mae = np.mean(np.abs(y_test_reg - y_pred_reg))

print(f"Оптимальные параметры модели: {best_gb_regressor_params}")
print(f"RMSE на тестовой выборке: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"Количество деревьев: {best_gb_regressor_params['n_estimators']}")
print(f"Скорость обучения: {best_gb_regressor_params['learning_rate']}")
print("\n=== Сводная информация ===")
print(f"Классификатор - Accuracy improvement: {(accuracy - best_gb_classifier_score):.4f}")
print(f"Регрессор - RMSE improvement: {(best_gb_regressor_score - rmse):.4f}")


=== Оценка лучшего Gradient Boosting Classifier ===
Оптимальные параметры модели: {'subsample': 0.8, 'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 4, 'learning_rate': 0.15}
Accuracy на тестовой выборке: 0.8944
F1-Score (weighted): 0.8962
Количество деревьев: 80
Скорость обучения: 0.15

=== Оценка лучшего Gradient Boosting Regressor ===
Оптимальные параметры модели: {'subsample': 0.8, 'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 5, 'loss': 'squared_error', 'learning_rate': 0.15}
RMSE на тестовой выборке: 4.4988
R² Score: 0.9215
MAE: 3.0811
MSE: 20.2389
Количество деревьев: 80
Скорость обучения: 0.15

=== Сводная информация ===
Классификатор - Accuracy improvement: 0.0240
Регрессор - RMSE improvement: 0.1826


Точность для классификации выросла ненамного, а вот для регрессии, кажется, это лучший результат среди всех рассмотренных моделей в работах.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию градиентного бустинга для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn. Будем использовать реализацию решающего дерева из ЛР №3.

In [34]:
import numpy as np
from tqdm import tqdm

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _mse(self, y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y)) ** 2)

    def _split(self, X, y, feature, threshold):
        mask = X[:, feature] < threshold
        return X[mask], y[mask], X[~mask], y[~mask]

    def _best_split(self, X, y):
        best_score = float('inf')
        best_split = None

        n_features = X.shape[1]
        for feature in range(n_features):
            # Берем только несколько случайных порогов для скорости
            unique_vals = np.unique(X[:, feature])
            if len(unique_vals) > 10:  # Ограничиваем количество порогов
                thresholds = np.random.choice(unique_vals, size=10, replace=False)
            else:
                thresholds = unique_vals
            
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                score = (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        n_samples = len(y)
        
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1 or
            n_samples == 0):
            return DecisionTreeNode(value=np.mean(y) if n_samples > 0 else 0)

        # Находим лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.mean(y))

        feature, threshold = best_split
        
        # Разделяем данные
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        
        # Если после разделения одна из частей слишком мала
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.mean(y))

        # Рекурсивно строим поддеревья
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(
            feature=feature,
            threshold=threshold,
            left=left_child,
            right=right_child
        )

    def fit(self, X, y):
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.float32)
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        return self._predict(x, node.right)

    def predict(self, X):
        X = np.array(X, dtype=np.float32)
        return np.array([self._predict(x, self.root) for x in X])

class CustomGradientBoostingClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, verbose=True):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.verbose = verbose
        self.models = []
        self.init_pred = None
        self.n_classes = None

    def _softmax(self, log_odds):
        exp_log_odds = np.exp(log_odds - np.max(log_odds, axis=1, keepdims=True))
        return exp_log_odds / np.sum(exp_log_odds, axis=1, keepdims=True)

    def fit(self, X, y):
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.int32)
        
        self.n_classes = len(np.unique(y))
        if self.n_classes < 2:
            raise ValueError("Должно быть как минимум 2 класса")
        
        # Инициализация предсказаний
        class_counts = np.bincount(y, minlength=self.n_classes)
        self.init_pred = np.log((class_counts + 1e-8) / len(y))
        
        # Начальные предсказания
        current_pred = np.tile(self.init_pred, (len(y), 1))
        self.models = []

        iterator = range(self.n_estimators)
        if self.verbose:
            iterator = tqdm(iterator, desc="GB Classifier Training")
        
        for i in iterator:
            probs = self._softmax(current_pred)
            
            # Вычисляем градиенты
            residuals = np.zeros_like(current_pred)
            for c in range(self.n_classes):
                y_mask = (y == c).astype(np.float32)
                residuals[:, c] = probs[:, c] - y_mask
            
            # Обучаем деревья для каждого класса
            trees = []
            for c in range(self.n_classes):
                tree = DecisionTreeRegressorCustom(
                    max_depth=self.max_depth,
                    min_samples_split=2,
                    min_samples_leaf=1
                )
                tree.fit(X, residuals[:, c])
                trees.append(tree)
            
            self.models.append(trees)
            
            # Обновляем предсказания
            for c, tree in enumerate(trees):
                current_pred[:, c] += self.learning_rate * tree.predict(X)
            
            if self.verbose and i % 5 == 0:
                current_acc = np.mean(np.argmax(probs, axis=1) == y)
                iterator.set_postfix({'acc': f'{current_acc:.3f}'})

    def predict_proba(self, X):
        X = np.array(X, dtype=np.float32)
        pred = np.tile(self.init_pred, (X.shape[0], 1))
        
        for trees in self.models:
            for c, tree in enumerate(trees):
                pred[:, c] += self.learning_rate * tree.predict(X)
        
        return self._softmax(pred)

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

Обучим модели и оценим их качество

In [35]:
# Обучаем кастомный классификатор градиентного бустинга
custom_gb_classifier = CustomGradientBoostingClassifier(
    n_estimators=10, 
    learning_rate=0.1, 
    max_depth=3,
    verbose=True
)

# Обучение модели
custom_gb_classifier.fit(X_train_class, y_train_class)

# Предсказания
y_pred_custom_class = custom_gb_classifier.predict(X_test_class)

# Оценка качества
accuracy_custom = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom = f1_score(y_test_class, y_pred_custom_class, average='weighted')

# Вывод результатов
print(f"\n=== Кастомный Gradient Boosting Classifier ===")
print(f"Accuracy: {accuracy_custom:.4f}")
print(f"F1-Score: {f1_custom:.4f}")

GB Classifier Training: 100%|██████████| 10/10 [00:14<00:00,  1.50s/it, acc=0.000]


=== Кастомный Gradient Boosting Classifier ===
Accuracy: 0.0056
F1-Score: 0.0026


In [38]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

# Создаем и обучаем кастомный регрессор
custom_gb_regressor = CustomGradientBoostingRegressor(
    n_estimators=10, 
    learning_rate=0.1, 
    max_depth=3,
    verbose=True
)

custom_gb_regressor.fit(X_train_reg, y_train_reg)

# Предсказания
y_pred_custom_reg = custom_gb_regressor.predict(X_test_reg)

# Метрики качества
rmse_custom = mean_squared_error(y_test_reg, y_pred_custom_reg, squared=False)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

# Вывод результатов
print(f"\n=== Кастомный Gradient Boosting Regressor ===")
print(f"RMSE: {rmse_custom:.4f}")
print(f"R² Score: {r2_custom:.4f}")

GB Regressor Training: 100%|██████████| 10/10 [00:00<00:00, 19.74it/s]


=== Кастомный Gradient Boosting Regressor ===
RMSE: 10.2992
R² Score: 0.5884


Модель классификации сравнительно долго работала, в отличии от регрессии, но и показала плохие результаты. Модель регрессии показала средние, но приемлимые результаты.